In [14]:
!pip install pulp
import pulp as pl
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import random
import tqdm
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
def generate_abcd():
  a = random.random()
  b = random.random()
  c = random.random()
  d = random.random()
  return a,b,c,d

notice $max\left\{x,c\right\} \equiv max\left\{x-c,0\right\} + c$, so a MLP with 1 hidden layer can generate function like $h(a,b,c) = t_1max\left\{w_{11}a+w_{12}b+w_{13}c + m_{1},0\right\} + n_{1} + t_2max\left\{w_{21}a+w_{22}b+w_{23}c + m_{2},0\right\} + n_{2}+t_3max\left\{w_{31}a+w_{32}b+w_{33}c + m_{3},0\right\} + n_{3} + t_4 = t_1max\left\{w_{11}a+w_{12}b+w_{13}c + m_{1}+p_1,p_1\right\} +t_2max\left\{w_{21}a+w_{22}b+w_{23}c + m_{2}+p_2,p2\right\} + t_3max\left\{w_{31}a+w_{32}b+w_{33}c + m_{3}+p_3,p3\right\} + n$

(Here $n = n_1+n_2+n_3+t_4-t_1p_1-t_2p_2-t_3p_3$)

It looks like the format of 3-agent situation.

In [16]:
H_function = nn.Sequential(
        nn.Linear(3, 3),
        nn.ReLU(),
        nn.Linear(3, 1)
        )

Try to optmize the ratio $\frac{h(a,b,c)+h(a,b,d)+h(a,c,d)+h(b,c,d)}{max\{ a+b+c+d,1\}}$, make it close to 3.
To make sure the ratio has a lower bound 3, if $ratio < 3$, we should do gradient ascent with a larger learning rate, else we should do gradient descent with a smaller learning rate.

In [45]:
nepoch = 3

In [42]:
#generate 3000 (a,b,c,d) as training data 
a = [0]*3000
b = [0]*3000
c = [0]*3000
d = [0]*3000
for i in range(3000):
  a[i], b[i], c[i], d[i] = generate_abcd()

In [30]:
def verify_lower_bound(w1,b1,w2,b2):
  model = pl.LpProblem(name="Verify-lower-bound", sense=pl.LpMinimize)
  a = pl.LpVariable(name='a',lowBound=0, upBound=1)
  b = pl.LpVariable(name='b',lowBound=0, upBound=1)
  c = pl.LpVariable(name='c',lowBound=0, upBound=1)
  d = pl.LpVariable(name='d',lowBound=0, upBound=1)
  # max{a+b+c+d,1}
  x0 = pl.LpVariable(name='x0')
  x0_ = pl.LpVariable(name='x0_',cat='Binary')
  model += (x0 >= 1, "constrain_01")
  model += (x0 >= a+b+c+d, "constrain_02")
  model += (x0 <= a+b+c+d+1000*x0_, "constrain_03")
  model += (x0 <= 1+1000*(1-x0_), "constrain_04")
  #h(a,b,c)
  x1 = pl.LpVariable(name='x1')
  x1_ = pl.LpVariable(name='x1_',cat='Binary')
  model += (x1 >= w1[0][0]*a + w1[0][1]*b + w1[0][2]*c + b1[0], "constrain_abc11")
  model += (x1 >= 0, "constain_abc12")
  model += (x1 <= w1[0][0]*a + w1[0][1]*b + w1[0][2]*c + b1[0] + 1000*x1_, "constrain_abc13")
  model += (x1 <= 1000*(1-x1_), "constrain_abc14")

  x2 = pl.LpVariable(name='x2')
  x2_ = pl.LpVariable(name='x2_',cat='Binary')
  model += (x2 >= w1[1][0]*a + w1[1][1]*b + w1[1][2]*c + b1[1], "constrain_abc21")
  model += (x2 >= 0, "constain_abc22")
  model += (x2 <= w1[1][0]*a + w1[1][1]*b + w1[1][2]*c + b1[1] + 1000*x2_, "constrain_abc23")
  model += (x2 <= 1000*(1-x2_), "constrain_abc24")

  x3 = pl.LpVariable(name='x3')
  x3_ = pl.LpVariable(name='x3_',cat='Binary')
  model += (x3 >= w1[2][0]*a + w1[2][1]*b + w1[2][2]*c + b1[2], "constrain_abc31")
  model += (x3 >= 0, "constain_abc32")
  model += (x3 <= w1[2][0]*a + w1[2][1]*b + w1[2][2]*c + b1[2] + 1000*x3_, "constrain_abc33")
  model += (x3 <= 1000*(1-x3_), "constrain_abc34")

  #h(a,b,d)
  y1 = pl.LpVariable(name='y1')
  y1_ = pl.LpVariable(name='y1_',cat='Binary')
  model += (y1 >= w1[0][0]*a + w1[0][1]*b + w1[0][2]*d + b1[0], "constrain_abd11")
  model += (y1 >= 0, "constain_abd12")
  model += (y1 <= w1[0][0]*a + w1[0][1]*b + w1[0][2]*d + b1[0] + 1000*y1_, "constrain_abd13")
  model += (y1 <= 1000*(1-y1_), "constrain_abd14")

  y2 = pl.LpVariable(name='y2')
  y2_ = pl.LpVariable(name='y2_',cat='Binary')
  model += (y2 >= w1[1][0]*a + w1[1][1]*b + w1[1][2]*d + b1[1], "constrain_abd21")
  model += (y2 >= 0, "constain_abd22")
  model += (y2 <= w1[1][0]*a + w1[1][1]*b + w1[1][2]*d + b1[1] + 1000*y2_, "constrain_abd23")
  model += (y2 <= 1000*(1-y2_), "constrain_abd24")

  y3 = pl.LpVariable(name='y3')
  y3_ = pl.LpVariable(name='y3_',cat='Binary')
  model += (y3 >= w1[2][0]*a + w1[2][1]*b + w1[2][2]*d + b1[2], "constrain_abd31")
  model += (y3 >= 0, "constain_abd32")
  model += (y3 <= w1[2][0]*a + w1[2][1]*b + w1[2][2]*d + b1[2] + 1000*y3_, "constrain_abd33")
  model += (y3 <= 1000*(1-y3_), "constrain_abd34")

  #h(a,c,d)
  z1 = pl.LpVariable(name='z1')
  z1_ = pl.LpVariable(name='z1_',cat='Binary')
  model += (z1 >= w1[0][0]*a + w1[0][1]*c + w1[0][2]*d + b1[0], "constrain_acd11")
  model += (z1 >= 0, "constain_acd12")
  model += (z1 <= w1[0][0]*a + w1[0][1]*c + w1[0][2]*d + b1[0] + 1000*z1_, "constrain_acd13")
  model += (z1 <= 1000*(1-z1_), "constrain_acd14")

  z2 = pl.LpVariable(name='z2')
  z2_ = pl.LpVariable(name='z2_',cat='Binary')
  model += (z2 >= w1[1][0]*a + w1[1][1]*c + w1[1][2]*d + b1[1], "constrain_acd21")
  model += (z2 >= 0, "constain_acd22")
  model += (z2 <= w1[1][0]*a + w1[1][1]*c + w1[1][2]*d + b1[1] + 1000*z2_, "constrain_acd23")
  model += (z2 <= 1000*(1-z2_), "constrain_acd24")

  z3 = pl.LpVariable(name='z3')
  z3_ = pl.LpVariable(name='z3_',cat='Binary')
  model += (z3 >= w1[2][0]*a + w1[2][1]*c + w1[2][2]*d + b1[2], "constrain_acd31")
  model += (z3 >= 0, "constain_acd32")
  model += (z3 <= w1[2][0]*a + w1[2][1]*c + w1[2][2]*d + b1[2] + 1000*z3_, "constrain_acd33")
  model += (z3 <= 1000*(1-z3_), "constrain_acd34")

  #h(b,c,d)
  q1 = pl.LpVariable(name='q1')
  q1_ = pl.LpVariable(name='q1_',cat='Binary')
  model += (q1 >= w1[0][0]*b + w1[0][1]*c + w1[0][2]*d + b1[0], "constrain_bcd11")
  model += (q1 >= 0, "constain_bcd12")
  model += (q1 <= w1[0][0]*b + w1[0][1]*c + w1[0][2]*d + b1[0] + 1000*q1_, "constrain_bcd13")
  model += (q1 <= 1000*(1-q1_), "constrain_bcd14")

  q2 = pl.LpVariable(name='q2')
  q2_ = pl.LpVariable(name='q2_',cat='Binary')
  model += (q2 >= w1[1][0]*b + w1[1][1]*c + w1[1][2]*d + b1[1], "constrain_bcd21")
  model += (q2 >= 0, "constain_bcd22")
  model += (q2 <= w1[1][0]*b + w1[1][1]*c + w1[1][2]*d + b1[1] + 1000*q2_, "constrain_bcd23")
  model += (q2 <= 1000*(1-q2_), "constrain_bcd24")

  q3 = pl.LpVariable(name='q3')
  q3_ = pl.LpVariable(name='q3_',cat='Binary')
  model += (q3 >= w1[2][0]*b + w1[2][1]*c + w1[2][2]*d + b1[2], "constrain_bcd31")
  model += (q3 >= 0, "constain_bcd32")
  model += (q3 <= w1[2][0]*b + w1[2][1]*c + w1[2][2]*d + b1[2] + 1000*q3_, "constrain_bcd33")
  model += (q3 <= 1000*(1-q3_), "constrain_bcd34")

  
  model += w2[0][0]*x1+w2[0][1]*x2+w2[0][2]*x3+b2[0]+w2[0][0]*y1+w2[0][1]*y2+w2[0][2]*y3+b2[0]+w2[0][0]*z1+w2[0][1]*z2+w2[0][2]*z3+b2[0]+w2[0][0]*q1+w2[0][1]*q2+w2[0][2]*q3+b2[0]-3*x0

  status = model.solve()

  print(f"status: {model.status}, {pl.LpStatus[model.status]}")
  print(f"objective: {model.objective.value()}")
  for var in model.variables():
    print(f"{var.name}: {var.value()}")
  for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")
  
  return


In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = H_function
model = model.to(device)
model.train()
for j in range(nepoch):
  loop = tqdm(range(3000))
  for i in loop:
    x1 = torch.tensor([a[j],b[j],c[j]]).to(device)
    x2 = torch.tensor([a[j],b[j],d[j]]).to(device)
    x3 = torch.tensor([a[j],c[j],d[j]]).to(device)
    x4 = torch.tensor([b[j],c[j],d[j]]).to(device)
    ratio = (model(x1)+model(x2)+model(x3)+model(x4))/torch.tensor([max(a[j]+b[j]+c[j]+d[j],1)]).to(device)
    if ratio < 3:
      optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, maximize = True)
    else:
      optimizer = torch.optim.SGD(model.parameters(), lr=1e-5, maximize = False, weight_decay=1e-4)
    optimizer.zero_grad()
    ratio.backward()
    optimizer.step()
    loop.set_description(f'Epoch [{j+1}/{nepoch}]')
    loop.set_postfix(ratio=ratio.item())
  w_layer0 = model.state_dict()["0.weight"].cpu().numpy()
  b_layer0 = model.state_dict()["0.bias"].cpu().numpy()
  w_layer2 = model.state_dict()["2.weight"].cpu().numpy()
  b_layer2 = model.state_dict()["2.bias"].cpu().numpy()

Epoch [3/3]: 100%|██████████| 3000/3000 [00:23<00:00, 125.57it/s, ratio=3.24]


In [47]:
print(w_layer0)
print(b_layer0)
print(w_layer2)
print(b_layer2)
verify_lower_bound(w_layer0,b_layer0,w_layer2,b_layer2)

[[ 0.18850572  0.918914    0.6833277 ]
 [ 0.03078114 -0.02010398  0.10714717]
 [-0.44438985  0.21058655 -0.02328267]]
[ 0.09928603  0.3685159  -0.14550172]
[[ 1.1184103  -0.11729623 -0.01210997]]
[0.4567645]
status: 1, Optimal
objective: -1.945793884546445
a: 1.0
b: 1.0
c: 1.0
d: 1.0
q1: 1.8900334
q1_: 0.0
q2: 0.48634024
q2_: 0.0
q3: 0.0
q3_: 1.0
x0: 4.0
x0_: 0.0
x1: 1.8900334
x1_: 0.0
x2: 0.48634024
x2_: 0.0
x3: 0.0
x3_: 1.0
y1: 1.8900334
y1_: 0.0
y2: 0.48634024
y2_: 0.0
y3: 0.0
y3_: 1.0
z1: 1.8900334
z1_: 0.0
z2: 0.48634024
z2_: 0.0
z3: 0.0
z3_: 1.0
constrain_01: 3.0
constrain_02: 0.0
constrain_03: 0.0
constrain_04: -997.0
constrain_abc11: -4.625234595323491e-08
constain_abc12: 1.8900334
constrain_abc13: -4.625234595323491e-08
constrain_abc14: -998.1099666
constrain_abc21: -3.3693409173807254e-09
constain_abc22: 0.48634024
constrain_abc23: -3.3693409173807254e-09
constrain_abc24: -999.51365976
constrain_abc31: 0.40258769132196903
constain_abc32: 0.0
constrain_abc33: -999.597412308678